# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['', '\r\n中天換照案聽證會剛結束，但NCC表示，聽證會中，中天著重於和NCC「攻防」，許多問題都沒有釐清，因此最快下周會邀請中天代表到每周三的例行委員會補充陳述。另外，前立委黃國昌晚間在臉書爆料「旺中微信群組截圖」，NCC官員指出，會持續蒐集資訊，這些群組截圖也會作為參考。', '\n', '\r\n中天換照案聽證會周一落幕，會中中天代表律師方伯勳針對違規案件數、裁罰不公等部分進行說明，但NCC官員表示，可能律師誤會當天是要「攻防戰」，關於聽證會設定的八大議題，包括是否已履行前次換照承諾、對過去營運不善情事未來如何確保改善、負責人是否影響新聞製播等，反而沒有完整說明。', '\n', '\r\n為了釐清所有議題，NCC擬再請中天代表到會說明，最快於下周三例行委員會邀請。官員表示，沒有設定來說明的代表人必須是神旺投資董事長蔡衍明，律師出席也可以；官員指出，中天代表在聽證會上提到有自請四位「鑑定人」，雖然與聽證制度不符，但NCC也歡迎這幾位專家學者提供意見。', '\n', '\r\nNCC發言人翁柏宗表示，將持續蒐集相關資資料。黃國昌晚間在個人粉絲團中張貼出疑似旺中集團管理階層的微信群組截圖，官員指出，這些資訊NCC「當然會列入參考」。', '\n', '\r\nNCC官員表示，聽證會周一結束後，就要等NCC委員會將該換照案排進議程、再次審理，之後才會再評估是否要開第二次聽證會或公聽會，並在中天執照到期日十二月十一日前，做出是否准許換照的決定。', '\n', '\n', '\n', '\n', '\n                    中天新聞台換照又有新進展。國家通訊傳播委員會（ＮＣＣ）昨天表示，由於中天新聞台在周一的聽證會中，著重於和ＮＣＣ「攻防」，...                  ', '\n                    「請大家驗明真象，還給我蔡衍明一個公道。這一切的爆料打壓手法，根本就是穿鑿附會！」針對時代力量前立委黃國昌今天晚間在臉書...                  ', '\n                    前立委黃國昌晚間在臉書爆料「旺中微信群組截圖」，流出內容提到民進黨前秘書長羅文嘉指旺中集團收受「對立政府」資助，群組內指...                  ', '\n             

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 0.558 seconds.


Prefix dict has been built successfully.


['中天 換照 案 聽證會 剛 結束 但 表示 聽證會 中 中天 著重 於 和 攻防 許多 問題 都 沒有 釐清 因此 最 快 下周 會 邀請 中天 代表 到 每 周三 的 例行 委員會 補充 陳述 另外 前立委 黃國昌 晚間 在 臉書 爆料 旺中 微信 群組 截圖 官員 指出 會 持續 蒐集 資訊 這些 群組 截圖 也 會 作為 參考 中天 換照 案 聽證會 周一 落幕 會 中 中天 代表 律師 方伯勳 針對 違規 案件數 裁罰 不公 等 部分 進行 說明 但 官員 表示 可能 律師 誤會 當天 是 要 攻防戰 關於 聽證會 設定 的 八大 議題 包括 是否 已 履行 前次 換照 承諾 對 過去 營運 不善 情事 未來 如何 確保 改善 負責人 是否 影響 新聞 製播 等 反而 沒有 完整 說明 為了 釐清 所有 議題 擬再 請 中天 代表 到 會 說明 最 快於 下 周三 例行 委員會 邀請 官員 表示 沒有 設定 來 說明 的 代表人 必須 是 神旺 投資 董事長 蔡衍明 律師 出席 也 可以 官員 指出 中天 代表 在 聽證會 上 提到 有 自請 四位 鑑定人 雖然 與 聽證 制度 不符 但 也 歡迎 這幾位 專家 學者 提供 意見 發言人 翁柏宗 表示 將 持續 蒐集 相關 資 資料 黃國昌 晚間 在 個 人 粉絲團 中 張貼 出 疑似 旺中 集團 管理 階層 的 微信 群組 截圖 官員 指出 這些 資訊 當然 會 列入 參考 官員 表示 聽證會 周一 結束 後 就 要 等 委員會 將該 換照 案 排進 議程 再次 審理 之後 才 會 再 評估 是否 要開 第二次 聽證會 或 公聽會 並在 中天 執照 到期日 十二月 十一日 前 做出 是否 准許 換照 的 決定 中天 新聞台 換照 又 有 新進展 國家 通訊 傳播 委員會 昨天 表示 由於 中天 新聞台 在 周一 的 聽證會 中 著重 於 和 攻防 請 大家 驗明 真象 還給 我 蔡衍明 一個 公道 這一 切 的 爆料 打壓 手法 根本 就是 穿鑿附會 針對 時代 力量 前立委 黃國昌 今天 晚間 在 臉書 前立委 黃國昌 晚間 在 臉書 爆料 旺中 微信 群組 截圖 流出 內容 提到 民進黨 前 秘書長 羅文嘉 指 旺中 集團 收受 對立 政府 資助 群組 內指 中天 換照 案 聽證會 剛 結束 但 表